In [1]:
pip install opencv-python cvzone autopy numpy


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import autopy
from cvzone.HandTrackingModule import HandDetector
import time

/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Initialize webcam
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

True

In [4]:
# Initialize hand detector
detector = HandDetector(detectionCon=0.8, maxHands=1)
screen_w, screen_h = autopy.screen.size()
click_down = False
plocX, plocY = 0, 0
clocX, clocY = 0, 0
smoothening = 7

I0000 00:00:1753333236.802734 45789001 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


In [ ]:
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    if hands:
        hand = hands[0]
        lmList = hand["lmList"]
        x1, y1 = lmList[8][0], lmList[8][1]   
        x2, y2 = lmList[6][0], lmList[6][1]   
        cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        screen_x = np.interp(x1, [100, 540], [0, screen_w])
        screen_y = np.interp(y1, [100, 380], [0, screen_h])
        clocX = plocX + (screen_x - plocX) / smoothening
        clocY = plocY + (screen_y - plocY) / smoothening
        clocX = max(0, min(screen_w - 1, clocX))
        clocY = max(0, min(screen_h - 1, clocY))
        autopy.mouse.move(clocX, clocY)
        plocX, plocY = clocX, clocY
        if y1 > y2 + 20:  
            if not click_down:
                autopy.mouse.click()
                click_down = True
                cv2.circle(img, (x1, y1), 15, (0, 255, 0), cv2.FILLED)
                cv2.putText(img, "Click!", (x1+20, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 3)
        else:
            click_down = False
    cv2.putText(img, "Virtual Mouse | Bend index finger to click | 'q' to quit", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    cv2.imshow("Virtual Mouse", img)
    
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1753333236.943413 45789233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753333237.015787 45789233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-07-24 10:30:37.839 Python[12178:45789001] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
/Users/ankitkumar/Library/Python/3.9/lib/python/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.war